In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('MNIST', one_hot=True)

# define batch set
batch_size = 100
# compute the num. of the batches
n_batch = mnist.train.num_examples // batch_size


def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean) # 平均值
        with tf.name_scope('stddv'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
    
# init weight
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape=shape, stddev=0.1) # 
    return tf.Variable(initial, name=name)

# init biases
def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape) # 
    return tf.Variable(initial, name=name)

# conv layer
def conv2d(x, W):
    # x input tensor of shape `[batch, in_height, in_weight, in_channels]`
    # W filter / kernel tensor of shape [filter_height, filter_Width, in_channels, out_channels]
    # `strides[0] = strider[3] = 1, strides[1] axis --> x, strides[2] --> y step
    # padding: A string from "SAME" or "VALID"
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# pooling layer
def max_pool_2x2(x):
    # ksize(1, x, y, 1)
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



#  define placeholder
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None, 784], name='x-input')
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')
    with tf.name_scope('x-image'):
        # change x shape --> 4D
        x_image = tf.reshape(x, shape=[-1, 28, 28, 1])

with tf.name_scope('Conv1'):        
    # init first conv layer 
    with tf.name_scope('W_conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32], name='W_conv1')# 5x5 sampling, 32个卷积核
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32], name='b_conv1')
    with tf.name_scope('relu'):
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    with tf.name_scope('h_pool1'):
        h_pool1 = max_pool_2x2(h_conv1)

# 2

with tf.name_scope('Conv1'):
    with tf.name_scope('W_conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64],name='W_conv2')# 5x5 sampling, 32个卷积核 --> 64
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64],name='b_conv2')
    with tf.name_scope('relu'):
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    with tf.name_scope('h_pool2'):
        h_pool2 = max_pool_2x2(h_conv2)
        

# draft for conpute
# -1 28 x 28 x 1
# -1 14 x 14 x 32
# -1 7 x 7 x 64
# fully-connect layer
with tf.name_scope('fc1'):
    with tf.name_scope('W_fc1'):
        W_fc1 = weight_variable([7*7*64, 1024],name='W_fc1')
    with tf.name_scope('b_fc1'):
        b_fc1 =bias_variable([1024],name='b_fc1')

    with tf.name_scope('h_pool2_flat'):
        # flatten operation 
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    with tf.name_scope('h_fc1'):
        # first fc output 
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    with tf.name_scope('keep_prob'):
        # keep_prob resp. probabilty of output enuron
        keep_prob = tf.placeholder(tf.float32)
    with tf.name_scope('h_fc_drop'):
        h_fc_drop = tf.nn.dropout(h_fc1, keep_prob)

# 2nd fully-connect layer
with tf.name_scope('fc1'):
    with tf.name_scope('W_fc2'):
        W_fc2 = weight_variable([1024, 10], name='W_fc2')
    with tf.name_scope('b_fc2'):
        b_fc2 =bias_variable([10], name='b_fc2')
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(tf.matmul(h_fc_drop, W_fc2) + b_fc2)

with tf.name_scope('cross_entropy'):
    # compute cross-entropy
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('cross_entropy',cross_entropy)

# adam optimizer
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



with tf.name_scope('accuracy'):
    # 结果保存在一个布尔列表中
    with tf.name_scope('correnct_prediction'):
        correnct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction, 1))
    # 求准确率
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correnct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

# 合并所有summary
merged = tf.summary.merge_all()
# training step


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('logs/train', sess.graph)
    test_writer = tf.summary.FileWriter('logs/test', sess.graph)

    for i in range(1001):
        # trining
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.5})
        # 记录训练集计算的参数
        summary = sess.run(merged, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
        train_writer.add_summary(summary, i)
        # 记录测试集计算的参数
        batch_xs,batch_ys = mnist.test.next_batch(batch_size)
        summary = sess.run(merged, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
        test_writer.add_summary(summary, i)

        if i % 100 == 0:
            test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images[:10000], y:mnist.test.labels[:10000], keep_prob:1.0})
            train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images[:10000], y:mnist.train.labels[:10000], keep_prob:1.0})
            print('Iter ' + str(i) + ', Testing Accuracy = ' + str(test_acc) + ', Training Accuracy = ' + str(train_acc))


        
   







Extracting MNIST\train-images-idx3-ubyte.gz
Extracting MNIST\train-labels-idx1-ubyte.gz
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz
Iter 0, Testing Accuracy = 0.1267, Training Accuracy = 0.125
Iter 100, Testing Accuracy = 0.5905, Training Accuracy = 0.5977
Iter 200, Testing Accuracy = 0.6531, Training Accuracy = 0.663
Iter 300, Testing Accuracy = 0.7075, Training Accuracy = 0.7128
Iter 400, Testing Accuracy = 0.9247, Training Accuracy = 0.9232
Iter 500, Testing Accuracy = 0.9372, Training Accuracy = 0.9373
Iter 600, Testing Accuracy = 0.9516, Training Accuracy = 0.9521
